In [ ]:
import gdown             

In [ ]:
gdown.download('https://drive.google.com/uc?id=1f4vOzkZfcqzEAo-yujV0KNpGXUsCeQ_W','test.kismet',quiet=False)

#https://drive.google.com/file/d/1f4vOzkZfcqzEAo-yujV0KNpGXUsCeQ_W/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=1f4vOzkZfcqzEAo-yujV0KNpGXUsCeQ_W
To: /content/test.kismet
100%|██████████| 13.8k/13.8k [00:00<00:00, 2.42MB/s]


'test.kismet'

In [ ]:
!mkdir -p ./data/chat_raw/whatsapp
!mv test.kismet /content/data/chat_raw/whatsapp

# **🤖 pistoBot**

> Create an AI that (try) to chat like you.<br>
> by [Simone Guardati](https://www.linkedin.com/in/simone-guardati/)


## 🥜 In a nutshell
1. Get your whatsapp and telegram data
2. Parse it to get a ML-like dataset
3. Train a GTP2 model
4. Chat with the model

**🔗 Resources**
- Chat parser - [github](https://github.com/pistocop/messaging-chat-parser)
- pistoBot - [github](https://github.com/pistocop/pistoBot)
- pistoBot website - [link](https://pistocop.github.io/pistoBot-website/)

<br>

**⚠️ Warning**
- It's always better not to run random scripts on personal information (like personal chat messages).
- I guarantee there is no double end, but you can always check (and use) the native code: <br>
[messaging-chat-parser](https://github.com/pistocop/messaging-chat-parser) and [pistobot](https://github.com/pistocop/pistoBot)


------
## 0️⃣ Init

In [ ]:
!nvidia-smi # p100 suggested

Tue Oct 12 01:54:42 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
!git clone --quiet https://github.com/pistocop/messaging-chat-parser.git
!pip install -q -r messaging-chat-parser/requirements.txt
!git clone --quiet https://github.com/pistocop/pistoBot.git

fatal: destination path 'messaging-chat-parser' already exists and is not an empty directory.
fatal: destination path 'pistoBot' already exists and is not an empty directory.


-----
## 1️⃣ Get the data
> Get your chat from whatsapp and telegram and upload them under `./messaging-chat-parser/data/chat_raw/` notebook folder


- **WhatsApp**
    - _.txt_ files exported from one or more chat - [how](https://faq.whatsapp.com/en/android/23756533/) (under "Export chat history" section)
        - don't export images
        - don't export group chats
    - place all txt files in `./messaging-chat-parser/data/chat_raw/whatsapp/*.txt`
- **Telegram** 
    - _.json_ with the telegram dump - [how](https://telegram.org/blog/export-and-more)
        - don't export images
        - choose "machine-readable JSON"
    - copy and rename the json file in `./messaging-chat-parser/data/chat_raw/telegram/telegram_dump.json`


------

## 2️⃣ Parse the data

**Whatsapp**
- Set the following variable `whatsapp_user_name`
    - Get it from one of the WhatsApp chat exported text. 
    <br> e.g. from one line: <br> 
     _12/12/19, 08:40 - `whatsapp_user_name`: bla bla bla_ 
- Datetime:
    - WhatsApp and Telegram have two different ways to manage the datetime.
    - Here are listed the two format, with Italian default values, if your data have different formats, change accordingly the next line values
        

In [ ]:
whatsapp_user_name = "Maddie_22" # <--- your name, extracted from Whatsapp data
whatsapp_datetime_format = "%m/%d/%y, %H:%M %p" # <-- American format used (MDY)

In [ ]:
# Whatsapp
print("> [WHATSAPP] start parsing...")
assert whatsapp_user_name is not None, "[!] Whatsapp user name not setted"
!cd messaging-chat-parser && python ./src/whatsapp_parser.py --session_token "<|endoftext|>" --user_name $whatsapp_user_name --time_format "$whatsapp_datetime_format"
print("> [WHATSAPP] parsing completed!\n\n")
print("----------------------------------")

# Join Telegram and Whatsapp data
!cd messaging-chat-parser/ && python ./src/joiner.py
training_data_lines = sum(1 for line in open('./messaging-chat-parser/data/chat_parsed/all-messages.txt'))
print(f"> [PARSER] Training file lines: {training_data_lines}")
print("----------------------------------")

# Check data size
if training_data_lines < 1000:
    print(f"[WARNING] attention insufficient training data ({training_data_lines} < 100K), it is recommended to export more chats")

------
## 3️⃣ Train a GTP2 model

- ⏳ The following cell could take **up to 10 hours**
    - An estimation of the total time will be prompted


In [ ]:
!cp ./messaging-chat-parser/data/chat_parsed/all-messages.txt ./pistoBot/data/inputs/chat_parsed/all-messages-endoftext.txt
!cd pistoBot/colab/ && bash run_training.sh gpt2-scratch

Installing common requirements...
     |████████████████████████████████| 269 kB 5.4 MB/s 
     |████████████████████████████████| 1.3 MB 33.7 MB/s 
     |████████████████████████████████| 495 kB 45.7 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 3.6 MB 32.1 MB/s 
     |████████████████████████████████| 404 kB 48.2 MB/s 
     |████████████████████████████████| 428 kB 42.3 MB/s 
     |████████████████████████████████| 58 kB 5.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires tornado~=5.1.0; python_version >= "3.0", but you have tornado 6.1 which is incompatible.
[gpt2-scratch model chosen]
Installing requirements...
     |████████████████████████████████| 248 kB 5.4 MB/s 
     |████████████████████████████████| 569 kB 

---
## 4️⃣ Chat with the model

Load the model

In [ ]:
from aitextgen import aitextgen
from pprint import pprint

files = os.listdir("./pistoBot/data/models_trained/")
files.remove('.gitkeep')
folder_name = files[0]

model_path = os.path.join(".", "pistoBot", "data", "models_trained", folder_name, "pytorch_model.bin")
config_path = os.path.join(".", "pistoBot", "data", "models_trained", folder_name,"config.json")
vocab_path = os.path.join(".", "pistoBot", "data", "models_trained", folder_name,"aitextgen-vocab.json")
merges_path = os.path.join(".", "pistoBot", "data", "models_trained", folder_name,"aitextgen-merges.txt")

ai = aitextgen(model=model_path, 
               config=config_path,
               vocab_file=vocab_path,
               merges_file=merges_path,
               to_gpu=True)

INFO:aitextgen:Constructing GPT-2 model from provided config.


ValueError: ignored

### 💬 Interactive mode
> Chat with the model one message at a time

- Run the following cell and use the prompt (✍) to write your messages
- The chats messages will show two tags:
    - **[others]** tags: messages wrote by the user
    - **[me]** tags: messages generated by the model

<br>

- Error _Max temperature reached_:
    - **Solution**: re-run the cell
    - Motivation: under the hood the program increase the _temperature_ value to get a new message that start with "[me]" tag. This is done until a max value is reached.


In [ ]:
chat = []
start_temperature = 0.7
max_temperature = 3.0

for _ in range(5):
    new_line = "[others] " + input("✍") + '\n'
    chat.append(new_line)
    
    me_token = False
    temperature = start_temperature
    input_network = ' '.join(chat)
    
    while not me_token:
        text = ai.generate(prompt=input_network, 
                           return_as_list=True, 
                           temperature=temperature)
        text = text[0] # batch of 1

        text = text.split('\n')
        chat_pos = len(chat)
        network_reply = text[chat_pos]

        if network_reply.startswith('[me]'):
            me_token = True
            network_reply = text[chat_pos] + '\n'
            chat.append(network_reply)
        else:
            if temperature >= max_temperature:
                raise RuntimeError("Max temperature reached")
            temperature += 0.1
    # print(f'temperature exit: {temperature}')
    print('Chat:')
    pprint(chat)
    print('---------------------')
    